In [15]:
import torch
import torchvision
import torchvision.transforms as transforms

In [16]:
tmptrainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True)
tmptestset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True)

Files already downloaded and verified
Files already downloaded and verified


In [17]:
# CIFAR10 데이터에서 훈련 고양이, 강아지 이미지 저장하기
CAT_VALUE = 3
DEER_VALUE = 4
DOG_VALUE = 5
for num, value in enumerate(tmptrainset):
    data, label = value
#     print(num, data, label)
    
    if(label == CAT_VALUE):
        data.save('./train_data/cat/%d_%d.jpeg'%(num, label))
    elif(label == DEER_VALUE):
        data.save('./train_data/deer/%d_%d.jpeg'%(num, label))
    elif(label == DOG_VALUE):
        data.save('./train_data/dog/%d_%d.jpeg'%(num, label))
        
for num, value in enumerate(tmptestset):
    data, label = value
#     print(num, data, label)
    
    if(label == CAT_VALUE):
        data.save('./test_data/cat/%d_%d.jpeg'%(num, label))
    elif(label == DEER_VALUE):
        data.save('./train_data/deer/%d_%d.jpeg'%(num, label))
    elif(label == DOG_VALUE):
        data.save('./test_data/dog/%d_%d.jpeg'%(num, label))

print('end!!')        

end!!


In [18]:
# 학습에 사용할 파라미터를 설정
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [19]:
# DataLoader 작업
# Windows 환경에서 BrokenPipeError가 발생한다면, torch.utils.data.DataLoader()의 num_worker를 0으로 설정해보세요.
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.ImageFolder(root='train_data/', transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=3)

testset = torchvision.datasets.ImageFolder(root='test_data/', transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=3)

classes = ('cat','deer','dog')

In [20]:
len(trainset), len(testset)

(15887, 2000)

In [21]:
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3,6,5),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(6,16,5),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        
        
        self.fc1 = nn.Sequential(
            nn.Linear(16*5*5, 120),
            nn.ReLU(),
            nn.Linear(120,2)
        )
        
    def forward(self, x):
        print(x.shape)
        out = self.layer1(x)
        print(out.shape)
        out = self.layer2(out)
        print(out.shape)
#         out = self.layer3(out)
#         print(out.shape)
        out = out.view(out.shape[0], -1)
        print(out.shape)
        out = self.fc1(out)
        return out

In [22]:
# Model 정의
device = 'cpu'
model = CNN()
#testing 
test_input = (torch.Tensor(100,3,32,32)).to(device) # 16, 13, 29
test_out = model(test_input)

torch.Size([100, 3, 32, 32])
torch.Size([100, 6, 14, 14])
torch.Size([100, 16, 5, 5])
torch.Size([100, 400])


In [23]:
import torch.nn as nn
import torch.nn.functional as F



class CNNDeep(nn.Module):
    def __init__(self):
        super(CNNDeep, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3,6,5),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(6,16,5),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.fc1 = nn.Sequential(
            nn.Linear(16*5*5, 120),
            nn.ReLU()
        )
        
        self.fc2 = nn.Sequential(
            nn.Linear(120, 60),
            nn.ReLU(),
            nn.Linear(60,3)
        )
        
        
    def forward(self, x):
#         print(x.shape)
        out = self.layer1(x)
#         print(out.shape)
        out = self.layer2(out)
#         print(out.shape)
        out = out.view(out.shape[0], -1)
#         print(out.shape)
        out = self.fc1(out)
#         print(out.shape)
        out = self.fc2(out)
        return out

In [24]:
# Model 정의
device = 'cpu'
model = CNNDeep()
#testing 
test_input = (torch.Tensor(100,3,32,32)).to(device) # 16, 13, 29
test_out = model(test_input)

In [25]:
import torch.optim as optim

loss_func  = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)
# optimizer = optim.SGD(model.parameters(), lr=0.005)

In [26]:
total_batch = len(trainloader)
print('총 배치의 수 : {}'.format(total_batch))

총 배치의 수 : 159


In [27]:
epochs = 5
for epoch in range(epochs):
    avg_cost = 0.0
    for num, data in enumerate(trainloader):
        imgs, labels = data
        imgs = imgs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        out = model(imgs)
        loss = loss_func(out, labels)
        loss.backward()
        optimizer.step()
        
        avg_cost += loss / total_batch
        
    print('[Epoch:{}] cost = {}'.format(epoch+1, avg_cost))
print('Learning Finished!')

[Epoch:1] cost = 0.9297338724136353
[Epoch:2] cost = 0.8514081239700317
[Epoch:3] cost = 0.8138974905014038
[Epoch:4] cost = 0.7914733290672302
[Epoch:5] cost = 0.7705807089805603
Learning Finished!


In [28]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 48 %
